english with indian accent

In [2]:
#!/usr/bin/env python3
"""
MetricGAN+ Fine-tuning - Hindi Male Dataset (Continued from Malayalam model)
OPTIMIZED FOR 3-HOUR TRAINING
"""

import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from speechbrain.pretrained import SpectralMaskEnhancement
import os
from pathlib import Path
from tqdm import tqdm
import tempfile
import shutil
from torch.cuda.amp import autocast, GradScaler
import random


# ============================================================
# Loss Function
# ============================================================
class SI_SNR_Loss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, estimate, target):
        if estimate.shape != target.shape:
            min_len = min(estimate.shape[-1], target.shape[-1])
            estimate = estimate[..., :min_len]
            target = target[..., :min_len]
        
        estimate = estimate - estimate.mean(dim=-1, keepdim=True)
        target = target - target.mean(dim=-1, keepdim=True)
        
        s_target = (torch.sum(estimate * target, dim=-1, keepdim=True) / 
                   (torch.sum(target ** 2, dim=-1, keepdim=True) + 1e-8)) * target
        e_noise = estimate - s_target
        
        si_snr = 10 * torch.log10(
            (torch.sum(s_target ** 2, dim=-1) + 1e-8) / 
            (torch.sum(e_noise ** 2, dim=-1) + 1e-8)
        )
        
        return -si_snr.mean()


# ============================================================
# Configuration
# ============================================================
class Config:
    # NEW HINDI DATASET PATHS
    noisy_train_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_male_converted\noisy_train_converted"
    clean_train_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_male_converted\clean_train_converted"
    noisy_val_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_male_converted\noisy_val_converted"
    clean_val_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_male_converted\clean_val_converted"
    
    # SAMPLING CONFIGURATION FOR 3-HOUR TRAINING
    max_train_files = 150  # Use 150 training files instead of 1800
    max_val_files = 30     # Use 30 validation files instead of 200
    
    # LOAD YOUR FINE-TUNED MALAYALAM MODEL
    pretrained_model = "speechbrain/metricgan-plus-voicebank"
    malayalam_checkpoint = r"J:\Hackathons\DEVSOC'26\checkpoints\metricgan_malayalam\best_model.ckpt"
    use_malayalam_checkpoint = True
    
    # NEW CHECKPOINT DIRECTORY FOR HINDI MODEL
    checkpoint_dir = r"J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_male"
    
    # OPTIMIZED FOR RTX 2050 + 3-HOUR TRAINING
    batch_size = 6
    num_epochs = 100
    learning_rate = 3e-5
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    sample_rate = 16000
    
    use_mixed_precision = True
    
    patience = 15
    min_delta = 0.001
    save_every_n_epochs = 10
    
    accumulation_steps = 2


config = Config()

print("="*60)
print("MetricGAN+ Fine-tuning on Hindi Male Speech")
print("="*60)
print(f"Device: {config.device}")
if config.device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Batch Size: {config.batch_size}")
    print(f"Effective Batch Size: {config.batch_size * config.accumulation_steps}")
    print(f"Mixed Precision: {config.use_mixed_precision}")
    print(f"Training on {config.max_train_files} files (sampled)")


# ============================================================
# Dataset - WITH SAMPLING FOR FASTER TRAINING
# ============================================================
class SpeechEnhancementDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, sample_rate=16000, max_files=None):
        self.noisy_dir = Path(noisy_dir)
        self.clean_dir = Path(clean_dir)
        self.sample_rate = sample_rate
        
        self.noisy_files = sorted(list(self.noisy_dir.glob("*.wav")))
        self.clean_files = sorted(list(self.clean_dir.glob("*.wav")))
        
        if set([f.name for f in self.noisy_files]) != set([f.name for f in self.clean_files]):
            raise ValueError("Filename mismatch!")
        
        total_files = len(self.noisy_files)
        
        # RANDOMLY SAMPLE FILES IF max_files IS SET
        if max_files is not None and len(self.noisy_files) > max_files:
            indices = random.sample(range(len(self.noisy_files)), max_files)
            self.noisy_files = [self.noisy_files[i] for i in sorted(indices)]
            self.clean_files = [self.clean_files[i] for i in sorted(indices)]
            print(f"  Sampled {len(self.noisy_files)} files from {total_files} total")
        else:
            print(f"  Loaded {len(self.noisy_files)} audio pairs")
    
    def __len__(self):
        return len(self.noisy_files)
    
    def __getitem__(self, idx):
        noisy, sr_n = torchaudio.load(str(self.noisy_files[idx]))
        clean, sr_c = torchaudio.load(str(self.clean_files[idx]))
        
        if sr_n != self.sample_rate:
            resampler = torchaudio.transforms.Resample(sr_n, self.sample_rate)
            noisy = resampler(noisy)
        if sr_c != self.sample_rate:
            resampler = torchaudio.transforms.Resample(sr_c, self.sample_rate)
            clean = resampler(clean)
        
        min_len = min(noisy.shape[1], clean.shape[1])
        noisy = noisy[:, :min_len]
        clean = clean[:, :min_len]
        
        return {
            'noisy': noisy.squeeze(0),
            'clean': clean.squeeze(0)
        }


# ============================================================
# Collate Function
# ============================================================
def collate_fn(batch):
    """Pad variable-length audio to same length"""
    noisy = [item['noisy'] for item in batch]
    clean = [item['clean'] for item in batch]
    
    max_len = max([n.shape[0] for n in noisy])
    
    noisy_padded = torch.zeros(len(noisy), max_len)
    clean_padded = torch.zeros(len(clean), max_len)
    
    for i, (n, c) in enumerate(zip(noisy, clean)):
        noisy_padded[i, :len(n)] = n
        clean_padded[i, :len(c)] = c
    
    return noisy_padded, clean_padded


# ============================================================
# Training Functions - OPTIMIZED WITH MIXED PRECISION
# ============================================================
def train_epoch(enhancer, dataloader, optimizer, criterion, device, scaler=None):
    enhancer.mods.enhance_model.train()
    total_loss = 0
    
    temp_dir = tempfile.mkdtemp()
    
    try:
        for batch_idx, (noisy, clean) in enumerate(tqdm(dataloader, desc="  Training", leave=False)):
            noisy = noisy.to(device)
            clean = clean.to(device)
            
            batch_size = noisy.shape[0]
            
            # Process each sample
            enhanced_batch = []
            for i in range(batch_size):
                temp_noisy = os.path.join(temp_dir, f"batch{batch_idx}_sample{i}.wav")
                torchaudio.save(temp_noisy, noisy[i:i+1].cpu(), config.sample_rate)
                
                enhanced = enhancer.enhance_file(temp_noisy)
                enhanced_batch.append(enhanced.to(device))
                
                os.remove(temp_noisy)
            
            enhanced_batch = torch.stack(enhanced_batch).unsqueeze(1)
            clean_batch = clean.unsqueeze(1)
            
            min_len = min(enhanced_batch.shape[-1], clean_batch.shape[-1])
            enhanced_batch = enhanced_batch[..., :min_len]
            clean_batch = clean_batch[..., :min_len]
            
            # Mixed precision training
            if config.use_mixed_precision and scaler is not None:
                with autocast():
                    loss = criterion(enhanced_batch, clean_batch)
                loss = loss / config.accumulation_steps
                
                scaler.scale(loss).backward()
                
                if (batch_idx + 1) % config.accumulation_steps == 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(enhancer.mods.parameters(), max_norm=5.0)
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
            else:
                loss = criterion(enhanced_batch, clean_batch)
                loss = loss / config.accumulation_steps
                loss.backward()
                
                if (batch_idx + 1) % config.accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(enhancer.mods.parameters(), max_norm=5.0)
                    optimizer.step()
                    optimizer.zero_grad()
            
            total_loss += loss.item() * config.accumulation_steps
        
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)
    
    return total_loss / len(dataloader)


def validate(enhancer, dataloader, criterion, device):
    enhancer.mods.enhance_model.eval()
    total_loss = 0
    
    temp_dir = tempfile.mkdtemp()
    
    try:
        with torch.no_grad():
            for batch_idx, (noisy, clean) in enumerate(tqdm(dataloader, desc="  Validating", leave=False)):
                noisy = noisy.to(device)
                clean = clean.to(device)
                
                batch_size = noisy.shape[0]
                
                enhanced_batch = []
                for i in range(batch_size):
                    temp_noisy = os.path.join(temp_dir, f"val{batch_idx}_sample{i}.wav")
                    torchaudio.save(temp_noisy, noisy[i:i+1].cpu(), config.sample_rate)
                    
                    enhanced = enhancer.enhance_file(temp_noisy)
                    enhanced_batch.append(enhanced.to(device))
                    
                    os.remove(temp_noisy)
                
                enhanced_batch = torch.stack(enhanced_batch).unsqueeze(1)
                clean_batch = clean.unsqueeze(1)
                
                min_len = min(enhanced_batch.shape[-1], clean_batch.shape[-1])
                enhanced_batch = enhanced_batch[..., :min_len]
                clean_batch = clean_batch[..., :min_len]
                
                loss = criterion(enhanced_batch, clean_batch)
                total_loss += loss.item()
    
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)
    
    return total_loss / len(dataloader)


# ============================================================
# Main
# ============================================================
def main():
    print("\n" + "="*60)
    print("Loading Model")
    print("="*60)
    
    enhancer = SpectralMaskEnhancement.from_hparams(
        source=config.pretrained_model,
        savedir="pretrained_models/metricgan-plus",
        run_opts={"device": config.device}
    )
    
    # Load Malayalam checkpoint if specified
    if config.use_malayalam_checkpoint and os.path.exists(config.malayalam_checkpoint):
        print(f"Loading Malayalam fine-tuned model from: {config.malayalam_checkpoint}")
        checkpoint = torch.load(config.malayalam_checkpoint, map_location=config.device)
        enhancer.mods.load_state_dict(checkpoint['model_state_dict'])
        print(f"✓ Loaded Malayalam model (trained for {checkpoint['epoch']+1} epochs)")
        print(f"  Previous val_loss: {checkpoint['val_loss']:.4f}")
    else:
        print("Starting from pretrained base model")
    
    # Unfreeze parameters
    for param in enhancer.mods.parameters():
        param.requires_grad = True
    
    trainable_params = sum(p.numel() for p in enhancer.mods.parameters() if p.requires_grad)
    print(f"✓ Trainable params: {trainable_params:,}")
    
    print("\n" + "="*60)
    print("Loading Hindi Male Datasets (SAMPLED)")
    print("="*60)
    
    print("Train dataset:")
    train_dataset = SpeechEnhancementDataset(
        config.noisy_train_dir, 
        config.clean_train_dir, 
        config.sample_rate,
        max_files=config.max_train_files
    )
    
    print("Validation dataset:")
    val_dataset = SpeechEnhancementDataset(
        config.noisy_val_dir, 
        config.clean_val_dir, 
        config.sample_rate,
        max_files=config.max_val_files
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config.batch_size, 
        shuffle=True, 
        num_workers=0,  # FIXED: Changed from 2 to 0 for Windows compatibility
        pin_memory=True,
        collate_fn=collate_fn
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=config.batch_size, 
        shuffle=False, 
        num_workers=0,  # FIXED: Changed from 2 to 0 for Windows compatibility
        pin_memory=True,
        collate_fn=collate_fn
    )
    
    print(f"\n✓ Train batches: {len(train_loader)} (from {config.max_train_files} files)")
    print(f"✓ Val batches: {len(val_loader)} (from {config.max_val_files} files)")
    
    criterion = SI_SNR_Loss().to(config.device)
    optimizer = torch.optim.AdamW(enhancer.mods.parameters(), lr=config.learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
    
    # Initialize mixed precision scaler
    scaler = GradScaler() if config.use_mixed_precision else None
    
    best_val_loss = float('inf')
    patience_counter = 0
    os.makedirs(config.checkpoint_dir, exist_ok=True)
    
    print("\n" + "="*60)
    print("Starting Fine-tuning on Hindi Dataset")
    print("="*60)
    
    for epoch in range(config.num_epochs):
        print(f"\nEpoch {epoch+1}/{config.num_epochs}")
        print("-" * 60)
        
        train_loss = train_epoch(enhancer, train_loader, optimizer, criterion, config.device, scaler)
        print(f"  Train Loss: {train_loss:.4f}")
        
        val_loss = validate(enhancer, val_loader, criterion, config.device)
        print(f"  Val Loss:   {val_loss:.4f}")
        
        scheduler.step(val_loss)
        print(f"  LR:         {optimizer.param_groups[0]['lr']:.6f}")
        
        if config.device == "cuda":
            print(f"  GPU Memory: {torch.cuda.max_memory_allocated() / 1e9:.2f}GB / {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f}GB")
            torch.cuda.reset_peak_memory_stats()
        
        if val_loss < best_val_loss - config.min_delta:
            best_val_loss = val_loss
            patience_counter = 0
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': enhancer.mods.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'config': vars(config)
            }, os.path.join(config.checkpoint_dir, "best_model.ckpt"))
            print(f"  ✓ Saved best model (val_loss: {val_loss:.4f})")
        else:
            patience_counter += 1
            print(f"  No improvement ({patience_counter}/{config.patience})")
        
        if (epoch + 1) % config.save_every_n_epochs == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': enhancer.mods.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss
            }, os.path.join(config.checkpoint_dir, f"checkpoint_epoch{epoch+1}.ckpt"))
            print(f"  ✓ Saved checkpoint: epoch {epoch+1}")
        
        if patience_counter >= config.patience:
            print(f"\n⚠ Early stopping triggered after {epoch+1} epochs")
            break
    
    print("\n" + "="*60)
    print("Fine-tuning Complete!")
    print("="*60)
    print(f"Best validation loss: {best_val_loss:.4f}")
    print(f"Model saved to: {config.checkpoint_dir}")
    print("\n✓ Ready for Hindi male audio enhancement!")


if __name__ == "__main__":
    main()


MetricGAN+ Fine-tuning on Hindi Male Speech
Device: cuda
GPU: NVIDIA GeForce RTX 2050
Batch Size: 6
Effective Batch Size: 12
Mixed Precision: True
Training on 150 files (sampled)

Loading Model
Loading Malayalam fine-tuned model from: J:\Hackathons\DEVSOC'26\checkpoints\metricgan_malayalam\best_model.ckpt
✓ Loaded Malayalam model (trained for 98 epochs)
  Previous val_loss: -17.1446
✓ Trainable params: 1,895,514

Loading Hindi Male Datasets (SAMPLED)
Train dataset:
  Sampled 150 files from 1800 total
Validation dataset:
  Sampled 30 files from 200 total

✓ Train batches: 25 (from 150 files)
✓ Val batches: 5 (from 30 files)

Starting Fine-tuning on Hindi Dataset

Epoch 1/100
------------------------------------------------------------


  Train Loss: -11.5270


  Val Loss:   -11.6906
  LR:         0.000030
  GPU Memory: 0.38GB / 4.29GB
  ✓ Saved best model (val_loss: -11.6906)

Epoch 2/100
------------------------------------------------------------


  Train Loss: -12.7521


  Val Loss:   -12.6105
  LR:         0.000030
  GPU Memory: 0.38GB / 4.29GB
  ✓ Saved best model (val_loss: -12.6105)

Epoch 3/100
------------------------------------------------------------


...

  Val Loss:   -15.3913
  LR:         0.000030
  GPU Memory: 0.37GB / 4.29GB
  No improvement (1/15)

Epoch 99/100
------------------------------------------------------------


  Train Loss: -17.0332


  Val Loss:   -15.3949
  LR:         0.000030
  GPU Memory: 0.37GB / 4.29GB
  ✓ Saved best model (val_loss: -15.3949)

Epoch 100/100
------------------------------------------------------------


  Train Loss: -17.0397


  Val Loss:   -15.3970
  LR:         0.000030
  GPU Memory: 0.38GB / 4.29GB
  ✓ Saved best model (val_loss: -15.3970)
  ✓ Saved checkpoint: epoch 100

Fine-tuning Complete!
Best validation loss: -15.3970
Model saved to: J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_male

✓ Ready for Hindi male audio enhancement!


hindi

In [5]:
#!/usr/bin/env python3
"""
MetricGAN+ Fine-tuning - Real Hindi (Hindi accent, Hindi content)
Starting from Hindi-male (English-with-Hindi-accent) model
"""

import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from speechbrain.pretrained import SpectralMaskEnhancement
import os
from pathlib import Path
from tqdm import tqdm
import tempfile
import shutil
from torch.cuda.amp import autocast, GradScaler
import random


# =========================
# SI-SNR Loss
# =========================
class SI_SNR_Loss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, estimate, target):
        if estimate.shape != target.shape:
            min_len = min(estimate.shape[-1], target.shape[-1])
            estimate = estimate[..., :min_len]
            target = target[..., :min_len]
        
        estimate = estimate - estimate.mean(dim=-1, keepdim=True)
        target = target - target.mean(dim=-1, keepdim=True)
        
        s_target = (torch.sum(estimate * target, dim=-1, keepdim=True) /
                    (torch.sum(target ** 2, dim=-1, keepdim=True) + 1e-8)) * target
        e_noise = estimate - s_target
        
        si_snr = 10 * torch.log10(
            (torch.sum(s_target ** 2, dim=-1) + 1e-8) /
            (torch.sum(e_noise ** 2, dim=-1) + 1e-8)
        )
        return -si_snr.mean()


# =========================
# Config
# =========================
class Config:
    # REAL HINDI (HINDI ACCENT, HINDI CONTENT) DATASET PATHS
    noisy_train_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\noisy_train"
    clean_train_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\clean_train"
    noisy_val_dir   = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\noisy_val"
    clean_val_dir   = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\clean_val"

    # SAMPLE SIZE TO KEEP TRAIN TIME REASONABLE
    max_train_files = 150
    max_val_files   = 30

    # START FROM YOUR HINDI-MALE CHECKPOINT
    pretrained_model = "speechbrain/metricgan-plus-voicebank"
    hindi_male_checkpoint = r"J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_male\best_model.ckpt"
    use_hindi_male_checkpoint = True

    # NEW CHECKPOINT DIR FOR REAL-HINDI MODEL
    checkpoint_dir = r"J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_augmented"

    # TRAINING SETUP
    batch_size = 6
    num_epochs = 100
    learning_rate = 2e-5   # a bit lower since we're fine-tuning on top of Hindi-male

    device = "cuda" if torch.cuda.is_available() else "cpu"
    sample_rate = 16000

    use_mixed_precision = True
    patience = 15
    min_delta = 0.001
    save_every_n_epochs = 10
    accumulation_steps = 2   # effective batch size = 12


config = Config()

print("=" * 60)
print("MetricGAN+ Fine-tuning on Hindi (Hindi accent, Hindi content)")
print("=" * 60)
print(f"Device: {config.device}")
if config.device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Batch Size: {config.batch_size}")
    print(f"Effective Batch Size: {config.batch_size * config.accumulation_steps}")
    print(f"Mixed Precision: {config.use_mixed_precision}")
    print(f"Training on {config.max_train_files} files (sampled)")


# =========================
# Dataset with Sampling
# =========================
class SpeechEnhancementDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, sample_rate=16000, max_files=None):
        self.noisy_dir = Path(noisy_dir)
        self.clean_dir = Path(clean_dir)
        self.sample_rate = sample_rate

        self.noisy_files = sorted(list(self.noisy_dir.glob("*.wav")))
        self.clean_files = sorted(list(self.clean_dir.glob("*.wav")))

        if set([f.name for f in self.noisy_files]) != set([f.name for f in self.clean_files]):
            raise ValueError("Filename mismatch between noisy and clean dirs")

        total_files = len(self.noisy_files)

        if max_files is not None and total_files > max_files:
            indices = random.sample(range(total_files), max_files)
            indices = sorted(indices)
            self.noisy_files = [self.noisy_files[i] for i in indices]
            self.clean_files = [self.clean_files[i] for i in indices]
            print(f"  Sampled {len(self.noisy_files)} files from {total_files} total")
        else:
            print(f"  Loaded {total_files} audio pairs")

    def __len__(self):
        return len(self.noisy_files)

    def __getitem__(self, idx):
        noisy, sr_n = torchaudio.load(str(self.noisy_files[idx]))
        clean, sr_c = torchaudio.load(str(self.clean_files[idx]))

        if sr_n != self.sample_rate:
            noisy = torchaudio.transforms.Resample(sr_n, self.sample_rate)(noisy)
        if sr_c != self.sample_rate:
            clean = torchaudio.transforms.Resample(sr_c, self.sample_rate)(clean)

        min_len = min(noisy.shape[1], clean.shape[1])
        noisy = noisy[:, :min_len]
        clean = clean[:, :min_len]

        return {"noisy": noisy.squeeze(0), "clean": clean.squeeze(0)}


# =========================
# Collate
# =========================
def collate_fn(batch):
    noisy = [b["noisy"] for b in batch]
    clean = [b["clean"] for b in batch]

    max_len = max(x.shape[0] for x in noisy)
    noisy_padded = torch.zeros(len(noisy), max_len)
    clean_padded = torch.zeros(len(clean), max_len)

    for i, (n, c) in enumerate(zip(noisy, clean)):
        noisy_padded[i, : len(n)] = n
        clean_padded[i, : len(c)] = c

    return noisy_padded, clean_padded


# =========================
# Train / Val Epochs
# =========================
def train_epoch(enhancer, dataloader, optimizer, criterion, device, scaler=None):
    enhancer.mods.enhance_model.train()
    total_loss = 0.0
    temp_dir = tempfile.mkdtemp()

    try:
        for batch_idx, (noisy, clean) in enumerate(tqdm(dataloader, desc="  Training", leave=False)):
            noisy = noisy.to(device)
            clean = clean.to(device)
            batch_size = noisy.shape[0]

            enhanced_batch = []
            for i in range(batch_size):
                temp_noisy = os.path.join(temp_dir, f"batch{batch_idx}_sample{i}.wav")
                torchaudio.save(temp_noisy, noisy[i:i+1].cpu(), config.sample_rate)
                enhanced = enhancer.enhance_file(temp_noisy)
                enhanced_batch.append(enhanced.to(device))
                os.remove(temp_noisy)

            enhanced_batch = torch.stack(enhanced_batch).unsqueeze(1)
            clean_batch = clean.unsqueeze(1)

            min_len = min(enhanced_batch.shape[-1], clean_batch.shape[-1])
            enhanced_batch = enhanced_batch[..., :min_len]
            clean_batch = clean_batch[..., :min_len]

            if config.use_mixed_precision and scaler is not None:
                with autocast():
                    loss = criterion(enhanced_batch, clean_batch)
                loss = loss / config.accumulation_steps
                scaler.scale(loss).backward()

                if (batch_idx + 1) % config.accumulation_steps == 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(enhancer.mods.parameters(), max_norm=5.0)
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
            else:
                loss = criterion(enhanced_batch, clean_batch)
                loss = loss / config.accumulation_steps
                loss.backward()

                if (batch_idx + 1) % config.accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(enhancer.mods.parameters(), max_norm=5.0)
                    optimizer.step()
                    optimizer.zero_grad()

            total_loss += loss.item() * config.accumulation_steps

    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)

    return total_loss / len(dataloader)


def validate(enhancer, dataloader, criterion, device):
    enhancer.mods.enhance_model.eval()
    total_loss = 0.0
    temp_dir = tempfile.mkdtemp()

    try:
        with torch.no_grad():
            for batch_idx, (noisy, clean) in enumerate(tqdm(dataloader, desc="  Validating", leave=False)):
                noisy = noisy.to(device)
                clean = clean.to(device)
                batch_size = noisy.shape[0]

                enhanced_batch = []
                for i in range(batch_size):
                    temp_noisy = os.path.join(temp_dir, f"val{batch_idx}_sample{i}.wav")
                    torchaudio.save(temp_noisy, noisy[i:i+1].cpu(), config.sample_rate)
                    enhanced = enhancer.enhance_file(temp_noisy)
                    enhanced_batch.append(enhanced.to(device))
                    os.remove(temp_noisy)

                enhanced_batch = torch.stack(enhanced_batch).unsqueeze(1)
                clean_batch = clean.unsqueeze(1)

                min_len = min(enhanced_batch.shape[-1], clean_batch.shape[-1])
                enhanced_batch = enhanced_batch[..., :min_len]
                clean_batch = clean_batch[..., :min_len]

                loss = criterion(enhanced_batch, clean_batch)
                total_loss += loss.item()
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)

    return total_loss / len(dataloader)


# =========================
# Main
# =========================
def main():
    print("\n" + "=" * 60)
    print("Loading Model")
    print("=" * 60)

    enhancer = SpectralMaskEnhancement.from_hparams(
        source=config.pretrained_model,
        savedir="pretrained_models/metricgan-plus",
        run_opts={"device": config.device},
    )

    if config.use_hindi_male_checkpoint and os.path.exists(config.hindi_male_checkpoint):
        print(f"Loading Hindi-male model from: {config.hindi_male_checkpoint}")
        checkpoint = torch.load(config.hindi_male_checkpoint, map_location=config.device)
        enhancer.mods.load_state_dict(checkpoint["model_state_dict"])
        print(f"✓ Loaded Hindi-male model (prev val_loss: {checkpoint['val_loss']:.4f})")
    else:
        print("Starting from base MetricGAN+ model")

    for p in enhancer.mods.parameters():
        p.requires_grad = True

    trainable_params = sum(p.numel() for p in enhancer.mods.parameters() if p.requires_grad)
    print(f"✓ Trainable params: {trainable_params:,}")

    print("\n" + "=" * 60)
    print("Loading Hindi Augmented Datasets (SAMPLED)")
    print("=" * 60)

    print("Train dataset:")
    train_dataset = SpeechEnhancementDataset(
        config.noisy_train_dir,
        config.clean_train_dir,
        config.sample_rate,
        max_files=config.max_train_files,
    )

    print("Validation dataset:")
    val_dataset = SpeechEnhancementDataset(
        config.noisy_val_dir,
        config.clean_val_dir,
        config.sample_rate,
        max_files=config.max_val_files,
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=0,   # Windows-safe
        pin_memory=True,
        collate_fn=collate_fn,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=collate_fn,
    )

    print(f"\n✓ Train batches: {len(train_loader)} (from {config.max_train_files} files)")
    print(f"✓ Val batches: {len(val_loader)} (from {config.max_val_files} files)")

    criterion = SI_SNR_Loss().to(config.device)
    optimizer = torch.optim.AdamW(enhancer.mods.parameters(), lr=config.learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=5, factor=0.5, verbose=True
    )

    scaler = GradScaler() if config.use_mixed_precision else None

    best_val_loss = float("inf")
    patience_counter = 0
    os.makedirs(config.checkpoint_dir, exist_ok=True)

    print("\n" + "=" * 60)
    print("Starting Fine-tuning on Hindi Augmented Dataset")
    print("=" * 60)

    for epoch in range(config.num_epochs):
        print(f"\nEpoch {epoch + 1}/{config.num_epochs}")
        print("-" * 60)

        train_loss = train_epoch(enhancer, train_loader, optimizer, criterion, config.device, scaler)
        print(f"  Train Loss: {train_loss:.4f}")

        val_loss = validate(enhancer, val_loader, criterion, config.device)
        print(f"  Val Loss:   {val_loss:.4f}")

        scheduler.step(val_loss)
        print(f"  LR:         {optimizer.param_groups[0]['lr']:.6f}")

        if config.device == "cuda":
            print(
                f"  GPU Memory: {torch.cuda.max_memory_allocated() / 1e9:.2f}GB / "
                f"{torch.cuda.get_device_properties(0).total_memory / 1e9:.2f}GB"
            )
            torch.cuda.reset_peak_memory_stats()

        if val_loss < best_val_loss - config.min_delta:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": enhancer.mods.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "train_loss": train_loss,
                    "val_loss": val_loss,
                    "config": vars(config),
                },
                os.path.join(config.checkpoint_dir, "best_model.ckpt"),
            )
            print(f"  ✓ Saved best model (val_loss: {val_loss:.4f})")
        else:
            patience_counter += 1
            print(f"  No improvement ({patience_counter}/{config.patience})")

        if (epoch + 1) % config.save_every_n_epochs == 0:
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": enhancer.mods.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "train_loss": train_loss,
                    "val_loss": val_loss,
                },
                os.path.join(config.checkpoint_dir, f"checkpoint_epoch{epoch + 1}.ckpt"),
            )
            print(f"  ✓ Saved checkpoint: epoch {epoch + 1}")

        if patience_counter >= config.patience:
            print(f"\n⚠ Early stopping triggered after {epoch + 1} epochs")
            break

    print("\n" + "=" * 60)
    print("Fine-tuning Complete!")
    print("=" * 60)
    print(f"Best validation loss: {best_val_loss:.4f}")
    print(f"Model saved to: {config.checkpoint_dir}")
    print("\n✓ Ready for Hindi-augmented audio enhancement!" )


MetricGAN+ Fine-tuning on Hindi (Hindi accent, Hindi content)
Device: cuda
GPU: NVIDIA GeForce RTX 2050
Batch Size: 6
Effective Batch Size: 12
Mixed Precision: True
Training on 150 files (sampled)


In [11]:
#!/usr/bin/env python3
"""
MetricGAN+ Fine-tuning - Hindi Augmented Dataset (Real Hindi, Hindi Accent)
Starting from: J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_male\best_model.ckpt
Dataset: J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset
"""

import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from speechbrain.pretrained import SpectralMaskEnhancement
import os
from pathlib import Path
from tqdm import tqdm
import tempfile
import shutil
from torch.cuda.amp import autocast, GradScaler
import random


# ============================================================
# Loss Function
# ============================================================
class SI_SNR_Loss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, estimate, target):
        if estimate.shape != target.shape:
            min_len = min(estimate.shape[-1], target.shape[-1])
            estimate = estimate[..., :min_len]
            target = target[..., :min_len]
        
        estimate = estimate - estimate.mean(dim=-1, keepdim=True)
        target = target - target.mean(dim=-1, keepdim=True)
        
        s_target = (torch.sum(estimate * target, dim=-1, keepdim=True) / 
                   (torch.sum(target ** 2, dim=-1, keepdim=True) + 1e-8)) * target
        e_noise = estimate - s_target
        
        si_snr = 10 * torch.log10(
            (torch.sum(s_target ** 2, dim=-1) + 1e-8) / 
            (torch.sum(e_noise ** 2, dim=-1) + 1e-8)
        )
        
        return -si_snr.mean()


# ============================================================
# Configuration
# ============================================================
class Config:
    # HINDI AUGMENTED DATASET PATHS
    noisy_train_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\noisy_train"
    clean_train_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\clean_train"
    noisy_val_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\noisy_val"
    clean_val_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\clean_val"
    
    # SAMPLING CONFIGURATION (adjust for time constraints)
    max_train_files = 150   # subset of training data
    max_val_files = 30      # subset of validation data
    
    # START FROM HINDI-MALE MODEL (ENGLISH WITH HINDI ACCENT)
    pretrained_model = "speechbrain/metricgan-plus-voicebank"
    hindi_male_checkpoint = r"J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_male\best_model.ckpt"
    use_hindi_male_checkpoint = True
    
    # NEW CHECKPOINT DIRECTORY FOR REAL HINDI
    checkpoint_dir = r"J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_augmented"
    
    # TRAINING HYPERPARAMETERS
    batch_size = 6
    num_epochs = 100
    learning_rate = 2e-5  # slightly lower since you are stacking fine-tuning steps
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    sample_rate = 16000
    
    use_mixed_precision = True
    
    patience = 15
    min_delta = 0.001
    save_every_n_epochs = 10
    
    accumulation_steps = 2  # effective batch size = batch_size * accumulation_steps


config = Config()

print("="*60)
print("MetricGAN+ Fine-tuning on Hindi Augmented Dataset (Real Hindi)")
print("="*60)
print(f"Device: {config.device}")
if config.device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Batch Size: {config.batch_size}")
    print(f"Effective Batch Size: {config.batch_size * config.accumulation_steps}")
    print(f"Mixed Precision: {config.use_mixed_precision}")
    print(f"Training on {config.max_train_files} files (sampled)")


# ============================================================
# Dataset - WITH SAMPLING FOR FASTER TRAINING
# ============================================================
class SpeechEnhancementDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, sample_rate=16000, max_files=None):
        self.noisy_dir = Path(noisy_dir)
        self.clean_dir = Path(clean_dir)
        self.sample_rate = sample_rate
        
        self.noisy_files = sorted(list(self.noisy_dir.glob("*.wav")))
        self.clean_files = sorted(list(self.clean_dir.glob("*.wav")))
        
        if set([f.name for f in self.noisy_files]) != set([f.name for f in self.clean_files]):
            raise ValueError("Filename mismatch between noisy and clean dirs!")
        
        total_files = len(self.noisy_files)
        
        if max_files is not None and len(self.noisy_files) > max_files:
            indices = random.sample(range(len(self.noisy_files)), max_files)
            self.noisy_files = [self.noisy_files[i] for i in sorted(indices)]
            self.clean_files = [self.clean_files[i] for i in sorted(indices)]
            print(f"  Sampled {len(self.noisy_files)} files from {total_files} total")
        else:
            print(f"  Loaded {len(self.noisy_files)} audio pairs")
    
    def __len__(self):
        return len(self.noisy_files)
    
    def __getitem__(self, idx):
        noisy, sr_n = torchaudio.load(str(self.noisy_files[idx]))
        clean, sr_c = torchaudio.load(str(self.clean_files[idx]))
        
        if sr_n != self.sample_rate:
            resampler = torchaudio.transforms.Resample(sr_n, self.sample_rate)
            noisy = resampler(noisy)
        if sr_c != self.sample_rate:
            resampler = torchaudio.transforms.Resample(sr_c, self.sample_rate)
            clean = resampler(clean)
        
        min_len = min(noisy.shape[1], clean.shape[1])
        noisy = noisy[:, :min_len]
        clean = clean[:, :min_len]
        
        return {
            'noisy': noisy.squeeze(0),
            'clean': clean.squeeze(0)
        }


# ============================================================
# Collate Function
# ============================================================
def collate_fn(batch):
    noisy = [item['noisy'] for item in batch]
    clean = [item['clean'] for item in batch]
    
    max_len = max([n.shape[0] for n in noisy])
    
    noisy_padded = torch.zeros(len(noisy), max_len)
    clean_padded = torch.zeros(len(clean), max_len)
    
    for i, (n, c) in enumerate(zip(noisy, clean)):
        noisy_padded[i, :len(n)] = n
        clean_padded[i, :len(c)] = c
    
    return noisy_padded, clean_padded


# ============================================================
# Training Functions
# ============================================================
def train_epoch(enhancer, dataloader, optimizer, criterion, device, scaler=None):
    enhancer.mods.enhance_model.train()
    total_loss = 0
    
    temp_dir = tempfile.mkdtemp()
    
    try:
        for batch_idx, (noisy, clean) in enumerate(tqdm(dataloader, desc="  Training", leave=False)):
            noisy = noisy.to(device)
            clean = clean.to(device)
            
            batch_size = noisy.shape[0]
            
            enhanced_batch = []
            for i in range(batch_size):
                temp_noisy = os.path.join(temp_dir, f"batch{batch_idx}_sample{i}.wav")
                torchaudio.save(temp_noisy, noisy[i:i+1].cpu(), config.sample_rate)
                
                enhanced = enhancer.enhance_file(temp_noisy)
                enhanced_batch.append(enhanced.to(device))
                
                os.remove(temp_noisy)
            
            enhanced_batch = torch.stack(enhanced_batch).unsqueeze(1)
            clean_batch = clean.unsqueeze(1)
            
            min_len = min(enhanced_batch.shape[-1], clean_batch.shape[-1])
            enhanced_batch = enhanced_batch[..., :min_len]
            clean_batch = clean_batch[..., :min_len]
            
            if config.use_mixed_precision and scaler is not None:
                with autocast():
                    loss = criterion(enhanced_batch, clean_batch)
                loss = loss / config.accumulation_steps
                
                scaler.scale(loss).backward()
                
                if (batch_idx + 1) % config.accumulation_steps == 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(enhancer.mods.parameters(), max_norm=5.0)
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
            else:
                loss = criterion(enhanced_batch, clean_batch)
                loss = loss / config.accumulation_steps
                loss.backward()
                
                if (batch_idx + 1) % config.accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(enhancer.mods.parameters(), max_norm=5.0)
                    optimizer.step()
                    optimizer.zero_grad()
            
            total_loss += loss.item() * config.accumulation_steps
        
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)
    
    return total_loss / len(dataloader)


def validate(enhancer, dataloader, criterion, device):
    enhancer.mods.enhance_model.eval()
    total_loss = 0
    
    temp_dir = tempfile.mkdtemp()
    
    try:
        with torch.no_grad():
            for batch_idx, (noisy, clean) in enumerate(tqdm(dataloader, desc="  Validating", leave=False)):
                noisy = noisy.to(device)
                clean = clean.to(device)
                
                batch_size = noisy.shape[0]
                
                enhanced_batch = []
                for i in range(batch_size):
                    temp_noisy = os.path.join(temp_dir, f"val{batch_idx}_sample{i}.wav")
                    torchaudio.save(temp_noisy, noisy[i:i+1].cpu(), config.sample_rate)
                    
                    enhanced = enhancer.enhance_file(temp_noisy)
                    enhanced_batch.append(enhanced.to(device))
                    
                    os.remove(temp_noisy)
                
                enhanced_batch = torch.stack(enhanced_batch).unsqueeze(1)
                clean_batch = clean.unsqueeze(1)
                
                min_len = min(enhanced_batch.shape[-1], clean_batch.shape[-1])
                enhanced_batch = enhanced_batch[..., :min_len]
                clean_batch = clean_batch[..., :min_len]
                
                loss = criterion(enhanced_batch, clean_batch)
                total_loss += loss.item()
    
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)
    
    return total_loss / len(dataloader)


# ============================================================
# Main
# ============================================================
def main():
    print("\n" + "="*60)
    print("Loading Model (Starting from Hindi-male fine-tuned)")
    print("="*60)
    
    enhancer = SpectralMaskEnhancement.from_hparams(
        source=config.pretrained_model,
        savedir="pretrained_models/metricgan-plus",
        run_opts={"device": config.device}
    )
    
    if config.use_hindi_male_checkpoint and os.path.exists(config.hindi_male_checkpoint):
        print(f"Loading Hindi-male model from: {config.hindi_male_checkpoint}")
        checkpoint = torch.load(config.hindi_male_checkpoint, map_location=config.device)
        enhancer.mods.load_state_dict(checkpoint['model_state_dict'])
        print(f"✓ Loaded Hindi-male model (previous val_loss: {checkpoint['val_loss']:.4f})")
    else:
        print("WARNING: Hindi-male checkpoint not found. Starting from base MetricGAN+.")
    
    for param in enhancer.mods.parameters():
        param.requires_grad = True
    
    trainable_params = sum(p.numel() for p in enhancer.mods.parameters() if p.requires_grad)
    print(f"✓ Trainable params: {trainable_params:,}")
    
    print("\n" + "="*60)
    print("Loading Hindi Augmented Datasets (SAMPLED)")
    print("="*60)
    
    print("Train dataset:")
    train_dataset = SpeechEnhancementDataset(
        config.noisy_train_dir, 
        config.clean_train_dir, 
        config.sample_rate,
        max_files=config.max_train_files
    )
    
    print("Validation dataset:")
    val_dataset = SpeechEnhancementDataset(
        config.noisy_val_dir, 
        config.clean_val_dir, 
        config.sample_rate,
        max_files=config.max_val_files
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config.batch_size, 
        shuffle=True, 
        num_workers=0,   # Windows-safe
        pin_memory=True,
        collate_fn=collate_fn
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=config.batch_size, 
        shuffle=False, 
        num_workers=0,   # Windows-safe
        pin_memory=True,
        collate_fn=collate_fn
    )
    
    print(f"\n✓ Train batches: {len(train_loader)} (from {config.max_train_files} files)")
    print(f"✓ Val batches: {len(val_loader)} (from {config.max_val_files} files)")
    
    criterion = SI_SNR_Loss().to(config.device)
    optimizer = torch.optim.AdamW(enhancer.mods.parameters(), lr=config.learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
    
    scaler = GradScaler() if config.use_mixed_precision else None
    
    best_val_loss = float('inf')
    patience_counter = 0
    os.makedirs(config.checkpoint_dir, exist_ok=True)
    
    print("\n" + "="*60)
    print("Starting Fine-tuning on Hindi Augmented Dataset")
    print("="*60)
    
    for epoch in range(config.num_epochs):
        print(f"\nEpoch {epoch+1}/{config.num_epochs}")
        print("-" * 60)
        
        train_loss = train_epoch(enhancer, train_loader, optimizer, criterion, config.device, scaler)
        print(f"  Train Loss: {train_loss:.4f}")
        
        val_loss = validate(enhancer, val_loader, criterion, config.device)
        print(f"  Val Loss:   {val_loss:.4f}")
        
        scheduler.step(val_loss)
        print(f"  LR:         {optimizer.param_groups[0]['lr']:.6f}")
        
        if config.device == "cuda":
            print(f"  GPU Memory: {torch.cuda.max_memory_allocated() / 1e9:.2f}GB / {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f}GB")
            torch.cuda.reset_peak_memory_stats()
        
        if val_loss < best_val_loss - config.min_delta:
            best_val_loss = val_loss
            patience_counter = 0
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': enhancer.mods.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'config': vars(config)
            }, os.path.join(config.checkpoint_dir, "best_model.ckpt"))
            print(f"  ✓ Saved best model (val_loss: {val_loss:.4f})")
        else:
            patience_counter += 1
            print(f"  No improvement ({patience_counter}/{config.patience})")
        
        if (epoch + 1) % config.save_every_n_epochs == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': enhancer.mods.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss
            }, os.path.join(config.checkpoint_dir, f"checkpoint_epoch{epoch+1}.ckpt"))
            print(f"  ✓ Saved checkpoint: epoch {epoch+1}")
        
        if patience_counter >= config.patience:
            print(f"\n⚠ Early stopping triggered after {epoch+1} epochs")
            break
    
    print("\n" + "="*60)
    print("Fine-tuning Complete!")
    print("="*60)
    print(f"Best validation loss: {best_val_loss:.4f}")
    print(f"Model saved to: {config.checkpoint_dir}")
    print("\n✓ Ready for Hindi augmented (real Hindi) audio enhancement!")


if __name__ == "__main__":
    main()


MetricGAN+ Fine-tuning on Hindi Augmented Dataset (Real Hindi)
Device: cuda
GPU: NVIDIA GeForce RTX 2050
Batch Size: 6
Effective Batch Size: 12
Mixed Precision: True
Training on 150 files (sampled)

Loading Model (Starting from Hindi-male fine-tuned)
Loading Hindi-male model from: J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_male\best_model.ckpt
✓ Loaded Hindi-male model (previous val_loss: -15.3970)
✓ Trainable params: 1,895,514

Loading Hindi Augmented Datasets (SAMPLED)
Train dataset:
  Sampled 150 files from 900 total
Validation dataset:
  Sampled 30 files from 98 total

✓ Train batches: 25 (from 150 files)
✓ Val batches: 5 (from 30 files)

Starting Fine-tuning on Hindi Augmented Dataset

Epoch 1/100
------------------------------------------------------------


  Train Loss: -10.9428


  Val Loss:   0.5862
  LR:         0.000020
  GPU Memory: 0.34GB / 4.29GB
  ✓ Saved best model (val_loss: 0.5862)

Epoch 2/100
------------------------------------------------------------


  Train Loss: -11.7364


  Val Loss:   -0.1671
  LR:         0.000020
  GPU Memory: 0.28GB / 4.29GB
  ✓ Saved best model (val_loss: -0.1671)

Epoch 3/100
------------------------------------------------------------


...

  Val Loss:   -3.5298
  LR:         0.000010
  GPU Memory: 0.28GB / 4.29GB
  No improvement (13/15)

Epoch 64/100
------------------------------------------------------------


  Train Loss: -16.5840


  Val Loss:   -3.5280
  LR:         0.000010
  GPU Memory: 0.28GB / 4.29GB
  No improvement (14/15)

Epoch 65/100
------------------------------------------------------------


  Train Loss: -16.5917


  Val Loss:   -3.5245
Epoch 00065: reducing learning rate of group 0 to 5.0000e-06.
  LR:         0.000005
  GPU Memory: 0.28GB / 4.29GB
  No improvement (15/15)

⚠ Early stopping triggered after 65 epochs

Fine-tuning Complete!
Best validation loss: -3.5497
Model saved to: J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_augmented

✓ Ready for Hindi augmented (real Hindi) audio enhancement!


In [10]:
from pathlib import Path

clean_val_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\clean_val"
noisy_val_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\noisy_val"

clean_val_dir = Path(clean_val_dir)
noisy_val_dir = Path(noisy_val_dir)

# Recompute sets (just to be safe)
clean_files = sorted([f for f in clean_val_dir.glob("*.wav")])
noisy_files = sorted([f for f in noisy_val_dir.glob("*.wav")])

clean_names = {f.name for f in clean_files}
noisy_names = {f.name for f in noisy_files}

only_in_clean = clean_names - noisy_names

print("Unmatched CLEAN files (will be deleted):")
for name in only_in_clean:
    print(" ", name)

# Delete them
for f in clean_files:
    if f.name in only_in_clean:
        print("Deleting:", f)
        f.unlink()

print("Done. clean_val now has only files that exist in noisy_val.")


Unmatched CLEAN files (will be deleted):
  hin_0190_snr20.wav
  hin_0164_snr5.wav
Deleting: J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\clean_val\hin_0164_snr5.wav
Deleting: J:\Hackathons\DEVSOC'26\dataset\final_training\data\hindi_augmented_dset\clean_val\hin_0190_snr20.wav
Done. clean_val now has only files that exist in noisy_val.


TAMIL

In [ ]:
#!/usr/bin/env python3
"""
MetricGAN+ Fine-tuning - Tamil Augmented Dataset
Building on: Malayalam → Hindi-English → Hindi-Augmented → TAMIL
Starting checkpoint: J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_augmented\best_model.ckpt
"""

import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from speechbrain.pretrained import SpectralMaskEnhancement
import os
from pathlib import Path
from tqdm import tqdm
import tempfile
import shutil
from torch.cuda.amp import autocast, GradScaler
import random


# ============================================================
# Loss Function
# ============================================================
class SI_SNR_Loss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, estimate, target):
        if estimate.shape != target.shape:
            min_len = min(estimate.shape[-1], target.shape[-1])
            estimate = estimate[..., :min_len]
            target = target[..., :min_len]
        
        estimate = estimate - estimate.mean(dim=-1, keepdim=True)
        target = target - target.mean(dim=-1, keepdim=True)
        
        s_target = (torch.sum(estimate * target, dim=-1, keepdim=True) / 
                   (torch.sum(target ** 2, dim=-1, keepdim=True) + 1e-8)) * target
        e_noise = estimate - s_target
        
        si_snr = 10 * torch.log10(
            (torch.sum(s_target ** 2, dim=-1) + 1e-8) / 
            (torch.sum(e_noise ** 2, dim=-1) + 1e-8)
        )
        
        return -si_snr.mean()


# ============================================================
# Configuration
# ============================================================
class Config:
    # TAMIL AUGMENTED DATASET PATHS
    noisy_train_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\tamil_augmented_dset\noisy_train"
    clean_train_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\tamil_augmented_dset\clean_train"
    noisy_val_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\tamil_augmented_dset\noisy_val"
    clean_val_dir = r"J:\Hackathons\DEVSOC'26\dataset\final_training\data\tamil_augmented_dset\clean_val"
    
    # SAMPLING CONFIGURATION FOR 3-HOUR TRAINING
    max_train_files = 150
    max_val_files = 30
    
    # START FROM HINDI AUGMENTED MODEL
    pretrained_model = "speechbrain/metricgan-plus-voicebank"
    hindi_augmented_checkpoint = r"J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_augmented\best_model.ckpt"
    use_hindi_augmented_checkpoint = True
    
    # NEW CHECKPOINT DIRECTORY FOR TAMIL
    checkpoint_dir = r"J:\Hackathons\DEVSOC'26\checkpoints\metricgan_tamil_augmented"
    
    # TRAINING HYPERPARAMETERS
    batch_size = 6
    num_epochs = 100
    learning_rate = 1.5e-5  # Even lower - stacking multiple fine-tuning steps
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    sample_rate = 16000
    
    use_mixed_precision = True
    
    patience = 15
    min_delta = 0.001
    save_every_n_epochs = 10
    
    accumulation_steps = 2


config = Config()

print("="*60)
print("MetricGAN+ Fine-tuning on Tamil Augmented Dataset")
print("="*60)
print(f"Device: {config.device}")
if config.device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Batch Size: {config.batch_size}")
    print(f"Effective Batch Size: {config.batch_size * config.accumulation_steps}")
    print(f"Mixed Precision: {config.use_mixed_precision}")
    print(f"Training on {config.max_train_files} files (sampled)")
print(f"Learning Rate: {config.learning_rate}")


# ============================================================
# Dataset
# ============================================================
class SpeechEnhancementDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, sample_rate=16000, max_files=None):
        self.noisy_dir = Path(noisy_dir)
        self.clean_dir = Path(clean_dir)
        self.sample_rate = sample_rate
        
        self.noisy_files = sorted(list(self.noisy_dir.glob("*.wav")))
        self.clean_files = sorted(list(self.clean_dir.glob("*.wav")))
        
        if len(self.noisy_files) != len(self.clean_files):
            raise ValueError(f"Different counts: {len(self.noisy_files)} noisy vs {len(self.clean_files)} clean")
        
        total_files = len(self.noisy_files)
        
        if max_files is not None and total_files > max_files:
            indices = sorted(random.sample(range(total_files), max_files))
            self.noisy_files = [self.noisy_files[i] for i in indices]
            self.clean_files = [self.clean_files[i] for i in indices]
            print(f"  Sampled {len(self.noisy_files)} pairs from {total_files} total")
        else:
            print(f"  Loaded {total_files} audio pairs")
    
    def __len__(self):
        return len(self.noisy_files)
    
    def __getitem__(self, idx):
        noisy_path = self.noisy_files[idx]
        clean_path = self.clean_files[idx]
        
        noisy, sr_n = torchaudio.load(str(noisy_path))
        clean, sr_c = torchaudio.load(str(clean_path))
        
        if sr_n != self.sample_rate:
            noisy = torchaudio.transforms.Resample(sr_n, self.sample_rate)(noisy)
        if sr_c != self.sample_rate:
            clean = torchaudio.transforms.Resample(sr_c, self.sample_rate)(clean)
        
        min_len = min(noisy.shape[1], clean.shape[1])
        noisy = noisy[:, :min_len]
        clean = clean[:, :min_len]
        
        return {
            "noisy": noisy.squeeze(0),
            "clean": clean.squeeze(0),
        }


# ============================================================
# Collate Function
# ============================================================
def collate_fn(batch):
    noisy = [item['noisy'] for item in batch]
    clean = [item['clean'] for item in batch]
    
    max_len = max([n.shape[0] for n in noisy])
    
    noisy_padded = torch.zeros(len(noisy), max_len)
    clean_padded = torch.zeros(len(clean), max_len)
    
    for i, (n, c) in enumerate(zip(noisy, clean)):
        noisy_padded[i, :len(n)] = n
        clean_padded[i, :len(c)] = c
    
    return noisy_padded, clean_padded


# ============================================================
# Training Functions
# ============================================================
def train_epoch(enhancer, dataloader, optimizer, criterion, device, scaler=None):
    enhancer.mods.enhance_model.train()
    total_loss = 0
    
    temp_dir = tempfile.mkdtemp()
    
    try:
        for batch_idx, (noisy, clean) in enumerate(tqdm(dataloader, desc="  Training", leave=False)):
            noisy = noisy.to(device)
            clean = clean.to(device)
            
            batch_size = noisy.shape[0]
            
            enhanced_batch = []
            for i in range(batch_size):
                temp_noisy = os.path.join(temp_dir, f"batch{batch_idx}_sample{i}.wav")
                torchaudio.save(temp_noisy, noisy[i:i+1].cpu(), config.sample_rate)
                
                enhanced = enhancer.enhance_file(temp_noisy)
                enhanced_batch.append(enhanced.to(device))
                
                os.remove(temp_noisy)
            
            enhanced_batch = torch.stack(enhanced_batch).unsqueeze(1)
            clean_batch = clean.unsqueeze(1)
            
            min_len = min(enhanced_batch.shape[-1], clean_batch.shape[-1])
            enhanced_batch = enhanced_batch[..., :min_len]
            clean_batch = clean_batch[..., :min_len]
            
            if config.use_mixed_precision and scaler is not None:
                with autocast():
                    loss = criterion(enhanced_batch, clean_batch)
                loss = loss / config.accumulation_steps
                
                scaler.scale(loss).backward()
                
                if (batch_idx + 1) % config.accumulation_steps == 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(enhancer.mods.parameters(), max_norm=5.0)
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
            else:
                loss = criterion(enhanced_batch, clean_batch)
                loss = loss / config.accumulation_steps
                loss.backward()
                
                if (batch_idx + 1) % config.accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(enhancer.mods.parameters(), max_norm=5.0)
                    optimizer.step()
                    optimizer.zero_grad()
            
            total_loss += loss.item() * config.accumulation_steps
        
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)
    
    return total_loss / len(dataloader)


def validate(enhancer, dataloader, criterion, device):
    enhancer.mods.enhance_model.eval()
    total_loss = 0
    
    temp_dir = tempfile.mkdtemp()
    
    try:
        with torch.no_grad():
            for batch_idx, (noisy, clean) in enumerate(tqdm(dataloader, desc="  Validating", leave=False)):
                noisy = noisy.to(device)
                clean = clean.to(device)
                
                batch_size = noisy.shape[0]
                
                enhanced_batch = []
                for i in range(batch_size):
                    temp_noisy = os.path.join(temp_dir, f"val{batch_idx}_sample{i}.wav")
                    torchaudio.save(temp_noisy, noisy[i:i+1].cpu(), config.sample_rate)
                    
                    enhanced = enhancer.enhance_file(temp_noisy)
                    enhanced_batch.append(enhanced.to(device))
                    
                    os.remove(temp_noisy)
                
                enhanced_batch = torch.stack(enhanced_batch).unsqueeze(1)
                clean_batch = clean.unsqueeze(1)
                
                min_len = min(enhanced_batch.shape[-1], clean_batch.shape[-1])
                enhanced_batch = enhanced_batch[..., :min_len]
                clean_batch = clean_batch[..., :min_len]
                
                loss = criterion(enhanced_batch, clean_batch)
                total_loss += loss.item()
    
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)
    
    return total_loss / len(dataloader)


# ============================================================
# Main
# ============================================================
def main():
    print("\n" + "="*60)
    print("Loading Model")
    print("="*60)
    print("Training lineage: Malayalam → Hindi-English → Hindi-Augmented → TAMIL")
    
    enhancer = SpectralMaskEnhancement.from_hparams(
        source=config.pretrained_model,
        savedir="pretrained_models/metricgan-plus",
        run_opts={"device": config.device}
    )
    
    if config.use_hindi_augmented_checkpoint and os.path.exists(config.hindi_augmented_checkpoint):
        print(f"\nLoading Hindi-Augmented model from:\n  {config.hindi_augmented_checkpoint}")
        checkpoint = torch.load(config.hindi_augmented_checkpoint, map_location=config.device)
        enhancer.mods.load_state_dict(checkpoint['model_state_dict'])
        print(f"✓ Loaded Hindi-Augmented model")
        print(f"  Trained for: {checkpoint['epoch']+1} epochs")
        print(f"  Previous val_loss: {checkpoint['val_loss']:.4f} dB")
    else:
        print("⚠️  WARNING: Hindi-Augmented checkpoint not found!")
        print("   Starting from base MetricGAN+ instead.")
    
    for param in enhancer.mods.parameters():
        param.requires_grad = True
    
    trainable_params = sum(p.numel() for p in enhancer.mods.parameters() if p.requires_grad)
    print(f"✓ Trainable params: {trainable_params:,}")
    
    print("\n" + "="*60)
    print("Loading Tamil Augmented Datasets")
    print("="*60)
    
    print("Train dataset:")
    train_dataset = SpeechEnhancementDataset(
        config.noisy_train_dir, 
        config.clean_train_dir, 
        config.sample_rate,
        max_files=config.max_train_files
    )
    
    print("Validation dataset:")
    val_dataset = SpeechEnhancementDataset(
        config.noisy_val_dir, 
        config.clean_val_dir, 
        config.sample_rate,
        max_files=config.max_val_files
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config.batch_size, 
        shuffle=True, 
        num_workers=0,
        pin_memory=True,
        collate_fn=collate_fn
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=config.batch_size, 
        shuffle=False, 
        num_workers=0,
        pin_memory=True,
        collate_fn=collate_fn
    )
    
    print(f"\n✓ Train batches: {len(train_loader)} (from {config.max_train_files} files)")
    print(f"✓ Val batches: {len(val_loader)} (from {config.max_val_files} files)")
    
    criterion = SI_SNR_Loss().to(config.device)
    optimizer = torch.optim.AdamW(enhancer.mods.parameters(), lr=config.learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
    
    scaler = GradScaler() if config.use_mixed_precision else None
    
    best_val_loss = float('inf')
    patience_counter = 0
    os.makedirs(config.checkpoint_dir, exist_ok=True)
    
    print("\n" + "="*60)
    print("Starting Fine-tuning on Tamil Dataset")
    print("="*60)
    
    for epoch in range(config.num_epochs):
        print(f"\nEpoch {epoch+1}/{config.num_epochs}")
        print("-" * 60)
        
        train_loss = train_epoch(enhancer, train_loader, optimizer, criterion, config.device, scaler)
        print(f"  Train Loss: {train_loss:.4f}")
        
        val_loss = validate(enhancer, val_loader, criterion, config.device)
        print(f"  Val Loss:   {val_loss:.4f}")
        
        scheduler.step(val_loss)
        print(f"  LR:         {optimizer.param_groups[0]['lr']:.6f}")
        
        if config.device == "cuda":
            print(f"  GPU Memory: {torch.cuda.max_memory_allocated() / 1e9:.2f}GB / {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f}GB")
            torch.cuda.reset_peak_memory_stats()
        
        if val_loss < best_val_loss - config.min_delta:
            best_val_loss = val_loss
            patience_counter = 0
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': enhancer.mods.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'config': vars(config),
                'training_history': 'Malayalam → Hindi-English → Hindi-Augmented → Tamil'
            }, os.path.join(config.checkpoint_dir, "best_model.ckpt"))
            print(f"  ✓ Saved best model (val_loss: {val_loss:.4f})")
        else:
            patience_counter += 1
            print(f"  No improvement ({patience_counter}/{config.patience})")
        
        if (epoch + 1) % config.save_every_n_epochs == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': enhancer.mods.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss
            }, os.path.join(config.checkpoint_dir, f"checkpoint_epoch{epoch+1}.ckpt"))
            print(f"  ✓ Saved checkpoint: epoch {epoch+1}")
        
        if patience_counter >= config.patience:
            print(f"\n⚠ Early stopping triggered after {epoch+1} epochs")
            break
    
    print("\n" + "="*60)
    print("Fine-tuning Complete!")
    print("="*60)
    print(f"Best validation loss: {best_val_loss:.4f} dB")
    print(f"Model saved to: {config.checkpoint_dir}")
    print("\n✓ Your model now supports: Malayalam, Hindi, AND Tamil! 🎉")
    print("   (Multilingual speech enhancement model)")


if __name__ == "__main__":
    main()


MetricGAN+ Fine-tuning on Tamil Augmented Dataset
Device: cuda
GPU: NVIDIA GeForce RTX 2050
Batch Size: 6
Effective Batch Size: 12
Mixed Precision: True
Training on 150 files (sampled)
Learning Rate: 1.5e-05

Loading Model
Training lineage: Malayalam → Hindi-English → Hindi-Augmented → TAMIL

Loading Hindi-Augmented model from:
  J:\Hackathons\DEVSOC'26\checkpoints\metricgan_hindi_augmented\best_model.ckpt
✓ Loaded Hindi-Augmented model
  Trained for: 50 epochs
  Previous val_loss: -3.5497 dB
✓ Trainable params: 1,895,514

Loading Tamil Augmented Datasets
Train dataset:
  Sampled 150 pairs from 900 total
Validation dataset:
  Sampled 30 pairs from 100 total

✓ Train batches: 25 (from 150 files)
✓ Val batches: 5 (from 30 files)

Starting Fine-tuning on Tamil Dataset

Epoch 1/100
------------------------------------------------------------


  Train Loss: -14.1822


  Val Loss:   -14.5889
  LR:         0.000015
  GPU Memory: 0.34GB / 4.29GB
  ✓ Saved best model (val_loss: -14.5889)

Epoch 2/100
------------------------------------------------------------


  Train Loss: -14.3861


  Val Loss:   -14.7292
  LR:         0.000015
  GPU Memory: 0.34GB / 4.29GB
  ✓ Saved best model (val_loss: -14.7292)

Epoch 3/100
------------------------------------------------------------


...

  Val Loss:   -15.3212
  LR:         0.000015
  GPU Memory: 0.34GB / 4.29GB
  No improvement (1/15)

Epoch 99/100
------------------------------------------------------------


  Train Loss: -15.9835


  Val Loss:   -15.3201
  LR:         0.000015
  GPU Memory: 0.34GB / 4.29GB
  No improvement (2/15)

Epoch 100/100
------------------------------------------------------------


  Train Loss: -15.9871


  Val Loss:   -15.3207
  LR:         0.000015
  GPU Memory: 0.34GB / 4.29GB
  No improvement (3/15)
  ✓ Saved checkpoint: epoch 100

Fine-tuning Complete!
Best validation loss: -15.3211 dB
Model saved to: J:\Hackathons\DEVSOC'26\checkpoints\metricgan_tamil_augmented

✓ Your model now supports: Malayalam, Hindi, AND Tamil! 🎉
   (Multilingual speech enhancement model)
